In [6]:
# Christian Gomez
# Professor Seth Berry 
# MSBR 70310 Section 02 
# February 23, 2025 

# ------Python Raw Code for Final Project ---------# 

# Importing libraries
import asyncio
import nest_asyncio
import smtplib
import pandas as pd
from email.message import EmailMessage
from playwright.async_api import async_playwright
from datetime import datetime
import pytz

# Email configuration
RECIPIENT_EMAIL = ["cgomez5@nd.edu"]
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
SENDER_EMAIL = "gomezchristian925@gmail.com"
SENDER_PASSWORD = "gwadmsxxfgusurru"  # Make sure to use environment variables instead

nest_asyncio.apply()  # Apply nest_asyncio for compatibility

# Scrape the Yahoo website 
async def scrape_yahoo_world_indices():
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=True)
        page = await browser.new_page()
        
        await page.goto("https://finance.yahoo.com/markets/world-indices/")
        await page.wait_for_load_state("domcontentloaded", timeout=60000)
        await asyncio.sleep(3)

        table = page.locator("div.tableContainer.yf-j24h8w table[data-testid='table-container']")
        await table.wait_for(state="visible", timeout=30000)
        rows = table.locator("tbody tr")
        row_count = await rows.count()

        data = []
        for i in range(row_count):
            row = rows.nth(i)
            cells = row.locator("td")
            cell_count = await cells.count()
            row_data = [ (await cells.nth(j).inner_text()).strip() for j in range(cell_count) ]
            data.append(row_data)
        
        await browser.close()
    return data

# Generate Yahoo Finance links
def generate_yahoo_link(symbol: str) -> str:
    encoded_symbol = symbol.replace('^', '%5E')
    return f"https://finance.yahoo.com/quote/{encoded_symbol}/"

# Main function
async def main():
    raw_data = await scrape_yahoo_world_indices()
    
    # Convert to DataFrame
    columns = ["Symbol", "Name", "Unused", "Price", "Change", "Change %", "Volume", "Day Range", "52 Wk Range"]
    df_all = pd.DataFrame(raw_data, columns=columns)
    df_all = df_all.drop(columns=["Unused", "Day Range", "52 Wk Range", "Volume"], errors="ignore")

    # Table 1 - USA Major Indices
    df_tickers = df_all[df_all["Symbol"].isin(["^GSPC", "^DJI", "^IXIC"])].copy()

    # Table 2 - Indices on the Move
    temp = df_all.copy()
    temp["Change % numeric"] = temp["Change %"].str.rstrip("%").astype(float)
    df_change = temp[(temp["Change % numeric"] > 0.75) | (temp["Change % numeric"] < -1.0)].copy()
    df_change = df_change.sort_values("Change % numeric", ascending=False)
    df_change = df_change.drop(columns=["Change % numeric"])

    # Add hyperlinks
    df_tickers["YahooLink"] = df_tickers["Symbol"].apply(generate_yahoo_link)
    df_change["YahooLink"] = df_change["Symbol"].apply(generate_yahoo_link)

    # Timestamp
    eastern = pytz.timezone("US/Eastern")
    display_timestamp = datetime.now(eastern).strftime("%Y-%m-%d %H:%M:%S")

    # Format hyperlinks
    def link_formatter(x):
        return f'<a href="{x}" target="_blank">{x}</a>' if pd.notna(x) else ""

    # Convert tables to HTML
    html_tickers = df_tickers.to_html(index=False, escape=False, formatters={"YahooLink": link_formatter})
    html_change  = df_change.to_html(index=False, escape=False, formatters={"YahooLink": link_formatter})

    # HTML content
    html_output = f"""
    <html>
    <head>
        <meta charset="utf-8">
        <title>World Indices Report - {display_timestamp}</title>
        <style>
        table {{
            border-collapse: collapse;
            width: 100%;
            margin-bottom: 20px;
        }}
        th, td {{
            border: 1px solid #dddddd;
            padding: 8px;
            text-align: left;
        }}
        th {{
            background-color: #f2f2f2;
        }}
        </style>
    </head>
    <body>
        <h1>World Indices Report</h1>
        <p>Date and Time (Eastern): {display_timestamp}</p>
        <h2>USA Tickers (^GSPC, ^DJI, ^IXIC)</h2>
        {html_tickers}
        <h2>Indices On The Move % (> 0.75% or < -1.0%)</h2>
        {html_change}
    </body>
    </html>
    """

    # Send email with report
    send_email_with_html(html_output)

# Function to send email
def send_email_with_html(html_content):
    eastern = pytz.timezone("US/Eastern")
    display_timestamp = datetime.now(eastern).strftime("%Y-%m-%d %H:%M:%S")

    msg = EmailMessage()
    msg["Subject"] = f"World Indices Report - {display_timestamp} (Eastern)"
    msg["From"] = SENDER_EMAIL
    msg["To"] = RECIPIENT_EMAIL
    msg.set_content("This email contains an HTML report.")
    msg.add_alternative(html_content, subtype="html")  # Embed HTML in email body

    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.send_message(msg)
        print("Email sent successfully")
    except Exception as e:
        print("Failed to send email:", e)

# Run main function
if __name__ == "__main__":
    asyncio.run(main())







Email sent successfully
